In [1]:
!pip install -U  langchain
!pip install -U langchain_community
!pip install -U langchain_openai
!pip install 'PyPDF2<3.0'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.8/222.8 kB 5.6 MB/s eta 0:00:00
  Attempting uninstall: PyPDF2
    Found existing installation: PyPDF2 3.0.1
    Uninstalling PyPDF2-3.0.1:
      Successfully uninstalled PyPDF2-3.0.1


In [32]:
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain.agents.agent_toolkits import create_retriever_tool
from langchain.agents.agent_toolkits import create_conversational_retrieval_agent
from langchain_openai.chat_models import ChatOpenAI
import requests
import PyPDF2
import io
import urllib.request

In [29]:
def download_remote_file(url, local_filename):
    response = requests.get(url)
    if response.status_code == 200:
        with open(local_filename, 'wb') as f:
            f.write(response.content)
        print(f"File downloaded and saved as {local_filename}")
    else:
        print("Failed to download file:", response.status_code)

In [41]:
URL = 'https://proceedings.mlr.press/v236/strieder24a/strieder24a.pdf'
req = urllib.request.Request(URL, headers={'User-Agent' : "Magic Browser"})
remote_file = urllib.request.urlopen(req).read()
remote_file_bytes = io.BytesIO(remote_file)
pdfdoc_remote = PyPDF2.PdfFileReader(remote_file_bytes)

text = ""
for i in range(pdfdoc_remote.numPages):
    text += pdfdoc_remote.getPage(i).extractText()

local_file_name = 'strieder24a.pdf'
with open(local_file_name, 'w') as f:
    f.write(text)

text_loader = TextLoader(local_file_name)
# Now you can use text_loader for language processing tasks
print("Sample research paper loaded successfully.")

os.environ["OPENAI_API_KEY"] ="your open api key. create one on openai.com"

# We load the document using LangChain’s handy extractors, formatters, loaders, embeddings, and LLMs
documents = text_loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)
print(texts)

Sample research paper loaded successfully.
[Document(page_content='Proceedings of Machine Learning Research vol 236:1–15, 2024 3rd Conference on Causal Learning and Reasoning\nDual Likelihood for Causal Inference under Structure Uncertainty\nDavid Strieder DAVID .STRIEDER @TUM .DEand Mathias Drton MATHIAS .DRTON @TUM .DE\nMunich Center for Machine Learning and Department of Mathematics, TUM School of Computation, Infor-\nmation and Technology, Technical University of Munich, Germany\nEditors: Francesco Locatello and Vanessa Didelez\nAbstract\nKnowledge of the underlying causal relations is essential for inferring the effect of interventions\nin complex systems. In a widely studied approach, structural causal models postulate noisy func-\ntional relations among interacting variables, where the underlying causal structure is then naturally\nrepresented by a directed graph whose edges indicate direct causal dependencies. In the typical\napplication, this underlying causal structure must b

In [43]:
# We use an OpenAI default embedding model
# Note the code in this example does not preserve privacy

embeddings = OpenAIEmbeddings()

# LangChain provides API functions to interact with FAISS

db = FAISS.from_documents(texts, embeddings)

# We create a 'retriever' that knows how to interact with our vector database using an augmented context
# We could construct the retriever ourselves from first principles but it's tedious
# Instead we'll use LangChain to create a retriever for our vector database

retriever = db.as_retriever()

tool = create_retriever_tool(
    retriever,
    "search_state_of_union",
    "Searches and returns documents regarding the state-of-the-union."
)
tools = [tool]

# We wrap an LLM (here OpenAI) with a conversational interface that can process augmented requests
# LangChain provides an API to interact with chat models

llm = ChatOpenAI(temperature = 0)
agent_executor = create_conversational_retrieval_agent(llm, tools, verbose=True)

input = "what is test inversion?"
result = agent_executor.invoke({"input": input})

# Response from the model

input = "What is dual likelihood?"
result = agent_executor.invoke({"input": input})

# Response from the model

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}